# Import Required Libraries and Modules
Import necessary libraries including os, time, tqdm, and specific modules from the project.

In [1]:
%cd ..

/home/user/llms


/home/user/.cache/pypoetry/virtualenvs/llms-rag-enx-6WuzWm9Q-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
#print cwd
import os
print(os.getcwd())

/home/user/llms


In [3]:
import os
import time
from tqdm import tqdm
from src.knowledge_graph import KnowledgeGraphBuilder, KnowledgeGraphIndex
from src.retrieval_utils import find_loader
from src.generation_utils import LLM_answer
from langchain_core.documents import Document

/home/user/.cache/pypoetry/virtualenvs/llms-rag-enx-6WuzWm9Q-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Process Documents
Load documents from a specified folder, process them to extract text, and split the text into chunks.

In [4]:
# Load the document that is in the specified folder
folder = 'data/mattermost/ENX @ FR/'  # Update the folder path as needed
start_time = time.time()
list_files = os.listdir(folder)
print(list_files)

# Sort the files alphabetically
list_files.sort()

# Generate full paths for the files
list_paths = [os.path.join(folder, file) for file in list_files]

total_chunks = []

# Process each file
for k in tqdm(range(len(list_paths))):
    # Find the appropriate loader for the file
    loader = find_loader(full_path=list_paths[k], name=list_files[k])
    
    # Load the text from the file
    text = loader.load()[0].page_content
    
    # Split the text into chunks of 1500 words
    list_chunks = [text[i:i+1500] for i in range(0, len(text), 1500)]
    
    # Reconstruct the langchain document objects from the chunks
    list_docs = [Document(page_content=chunk) for chunk in list_chunks]
    
    # Extend the total chunks with the newly created document chunks
    total_chunks.extend(list_docs)

["Dialogue_du_28_June_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_12_April_2021_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_19_October_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_12_June_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_19_November_2021_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_29_September_2022_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_25_March_2024_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_09_June_2022_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_09_February_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_11_September_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_24_October_2023_sur_la_plateforme_Mattermost_dans_la_channel_'ENX_@_FR'.md", "Dialogue_du_07_

100%|██████████| 750/750 [00:05<00:00, 133.06it/s]


# Build the Knowledge Graph
Use the processed document chunks to build a knowledge graph and index it for querying.

In [5]:
# Import the KnowledgeGraphIndex
kg_index = KnowledgeGraphIndex(disambiguate_threshold=60)

# Build the knowledge graph from the document chunks
db = kg_index.from_documents(documents=total_chunks, overwrite=True)

# Retrieve the knowledge graph
kg = kg_index.get_graph()

# Print all existing nodes in the knowledge graph
print("NODES:", kg.nodes())

Extracting knowledge relations:   0%|          | 0/831 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Extracting knowledge relations:   1%|          | 9/831 [00:01<01:59,  6.85it/s]

Error in triplet: ('juliette.spinnato ', ' works_with', 'juliette.spinnato') while merging subject with entity: juliettespinnato


Extracting knowledge relations:   2%|▏         | 17/831 [00:02<01:25,  9.54it/s]

Error in triplet: ('Myriam ', ' works_with', 'Myriam') while merging subject with entity: Myriam
Error in triplet: ('Myriam ', ' works_for', 'Myriam') while merging subject with entity: Myriam


Extracting knowledge relations:   2%|▏         | 19/831 [00:03<01:47,  7.56it/s]

Error in triplet: ('NovaAid ', ' works_with', 'NovaAid') while merging subject with entity: NovaAid


Extracting knowledge relations:   2%|▏         | 20/831 [00:03<01:56,  6.97it/s]

# Perform Question Answering with the Knowledge Graph
Perform a similarity search on the knowledge graph to find relevant documents and use a language model to answer a specific question based on the knowledge graph context.

In [ ]:
query = "alice"
found_docs = db.similarity_search(query=query, k=10)

related_entities = []
for k in found_docs:
    related_entities.append(k.page_content)

print("RELATED ENTITIES:", related_entities)
    
    

# Display found documents
# print("FOUND DOCS:", found_docs)


Batches: 100%|██████████| 1/1 [00:00<00:00, 71.25it/s]

RELATED ENTITIES: ['alicemartzloff', 'femmes', 'discussions', 'Find a time', 'people', 'supports', 'womaninlotusposition', 'Assessment', 'lycéennes', 'éléments textuels']


# Try entity disambiguation

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:


def cosine_sim_entity(e1, e2):
    """
    Compute cosine similarity between two entities embeddings.
    """
    # Assuming `model` is defined elsewhere and has an `encode` method
    e1_embedding = model.encode(e1,show_progress_bar=False).reshape(1, -1)  # Reshape to 2D array
    e2_embedding = model.encode(e2,show_progress_bar=False).reshape(1, -1)  # Reshape to 2D array
    
    # Compute cosine similarity between e1 and e2 embeddings
    return cosine_similarity(e1_embedding, e2_embedding)[0][0]


print(cosine_sim_entity("Aziz Jedidi", "aziz.jedidi"))

0.8762475


In [ ]:
from thefuzz import fuzz

entity="euranova france"
for k in related_entities:
    ratio=fuzz.token_set_ratio(entity, k)
    cosine_sim=cosine_sim_entity(entity, k)
    print("RATIO between", entity, "and", k, "is", ratio)
    print("COSINE SIMILARITY between", entity, "and", k, "is", cosine_sim)

RATIO between euranova france and euranova is 100
COSINE SIMILARITY between euranova france and euranova is 0.6347387
RATIO between euranova france and knowledgeeuranovaeu is 53
COSINE SIMILARITY between euranova france and knowledgeeuranovaeu is 0.34978235
RATIO between euranova france and researcheuranovaeucategorypublications is 42
COSINE SIMILARITY between euranova france and researcheuranovaeucategorypublications is 0.19227621
RATIO between euranova france and discussions is 23
COSINE SIMILARITY between euranova france and discussions is 0.13835481
RATIO between euranova france and Find a time is 38
COSINE SIMILARITY between euranova france and Find a time is 0.07578041
RATIO between euranova france and supports is 26
COSINE SIMILARITY between euranova france and supports is 0.1380426
RATIO between euranova france and http100131025000 is 0
COSINE SIMILARITY between euranova france and http100131025000 is 0.16779056
RATIO between euranova france and eu is 24
COSINE SIMILARITY betwe

In [ ]:

# Get the first document object
doc = found_docs[0].page_content + str(found_docs[0].metadata)

# Create final prompt using context and query
final_prompt = "Answer the following question: " + query + " using the following knowledge graph context: " + doc

# Display the final prompt
print("Final Prompt:", final_prompt)

# Formulate the answer to the question using a language model
answer = LLM_answer(prompt=final_prompt, model_name='gemma2', llm_provider='ollama', stream=False)

# Display the answer
print("ANSWER:", answer)

Final Prompt: Answer the following question: Qui sont les personnes avec qui DucTang Hoang travaille, donne la liste ? using the following knowledge graph context: ducthanghoang{'relation_list': [{'relation': 'workswith', 'linked_entity': 'screen'}, {'relation': 'includedin', 'linked_entity': 'SynoCli Network Tools'}, {'relation': 'workswith', 'linked_entity': 'ismailbachchar'}, {'relation': 'makesuseof', 'linked_entity': 'open space'}, {'relation': 'workswith', 'linked_entity': 'tristanschreiber'}, {'relation': 'includedin', 'linked_entity': 'bpost'}, {'relation': 'contributesto', 'linked_entity': 'ThorOdinFenrir'}, {'relation': 'workswith', 'linked_entity': 'nvidiasmi'}, {'relation': 'worksfor', 'linked_entity': 'mdp'}, {'relation': 'makesuseof', 'linked_entity': 'NVME'}, {'relation': 'makesuseof', 'linked_entity': 'home'}, {'relation': 'workswith', 'linked_entity': 'tmux'}, {'relation': 'worksfor', 'linked_entity': 'alikaraouzene'}, {'relation': 'makesuseof', 'linked_entity': 'Conve

# (Optional) Extract Subgraph and Perform Cypher Query
Optionally, demonstrate how to extract a subgraph based on specific entities and perform a Cypher query on the knowledge graph.

In [ ]:
# Extract subgraph based on specific entities
entities_to_extract = ['BioTech', 'EURA NOVA']
subgraph = kg_index.extract_subgraph(entities=entities_to_extract)
print("SUBGRAPH:", subgraph)

# Perform a Cypher query on the knowledge graph
cypher_query = """
MATCH (A)<-[]-(B)
WHERE A.entity_name = "BioTech"
RETURN B.entity_name
"""
result = kg_index.query_graph(cypher_query)
print("CYPHER QUERY RESULT:", result)

NetworkXError: The node BioTech is not in the digraph.